In [ ]:



importlib.reload(vp)
importlib.reload(load_clean_lib)
import sys
import time

# record the code duration
start_time = time.time()

# loop through all dates
for current_date in od["date"].unique():
    # Current day asset data
    
    # select current date data
    od_date = od[od["date"] == current_date]
    FW_date = FW[FW["date"] == current_date]
    ZCY_date = ZCY_curves[ZCY_curves['date'] == current_date]

    tickers_date = od_date["ticker"].unique()

    # Current day Zero Cupon curve
    if ZCY_date.empty:
        print(f"No yield data available: {current_date} (interpolating)")
        ZCY_date = vp.interpolate_yield_curve_between_dates(ZCY_curves, current_date)

    # loop through all tickers for the date
    for current_ticker in tickers_date:        
        # two functions to get and set info in summary_dly_df
        def get(info):
            return summary_dly_df.at[(current_ticker, current_date), info]        
        def set(info, value, reason = None):
            summary_dly_df.at[(current_ticker, current_date), info] = value
            if reason is not None:
                summary_dly_df.at[(current_ticker, current_date), "Inactive reason"] = reason

        # select current ticker date data
        od_ticker = od_date[od_date["ticker"] == current_ticker]
        FW_ticker = FW_date[FW_date["ticker"] == current_ticker]
        days_ticker = od_ticker["days"].unique()
        set("#days", value = len(days_ticker)) # report number of unique dates for ticker on date
        
        # remove tickers that has less than two unique dates
        if len(days_ticker) < 2:
            set("Active", value = False, reason = "unique(_days) < 2") # False
            # print(f"skipped {current_ticker} for {current_date} due to no data available.")
            continue

        # remove tickers that has no dates within 90 days til expiration        
        if min(days_ticker) > 90:
            set("Active", value = False, reason = "min days > 90") # False
            print(f"skipped {current_ticker} for {current_date} due to min(days) > 90")
            continue
        
        # select two shortest dates that are above 7 days (skip first only if multiple)
        days_ticker = sorted(days_ticker)
        low_2_days = days_ticker[:2]
        if low_2_days[0] <= 8:
            if len(days_ticker) < 3:
                set("Active", value = False, reason = "min days <= 8 & len < 3") # False
                continue
            low_2_days = days_ticker[1:3]
        
        # report two selected days
        set("low days", value = low_2_days[0])
        set("high days", value = low_2_days[1])
        
        # report current date to be active for current ticker, if at least 3 unique strikes. Otherwise inactive
        set("Active", value = True, reason = "")
        for days, high_low in zip(low_2_days, ["low", "high"]): # loops through the highest and lowest days til expiration. e.g. 15 days ("low") and 32 days ("high").
            number_of_K = (od_ticker[od_ticker["days"] == days]["strike_price"]).nunique()
            set(f"{high_low} #K", value = number_of_K)
            if number_of_K < 3:
                set("Active", value = False, reason = f"unique(K) < 3")
        if not get("Active"):
            continue   # if not active
        
        # loop through the two selected dates
        for days in low_2_days:
            TTM = days / 365.0
            # if days > 100: break
            
            # select TTM data (only the given days til expiration)
            od_TTM = od_ticker[od_ticker["days"] == days]
            
            # interpolate the interest rate and forward rate
            r = vp.interpolated_ZCY(ZCY_date, days)
            F = vp.interpolated_FW(FW_ticker, days)
            
            # IV = T_data["IV"]["impl_volatility"] #smile, would need to be interpolated            
            # K = T_data["od"]["strike_price"]
            # IV = T_data["od"]["impl_volatility"]
            # cp_flag = T_data["od"]["cp_flag"]
            # option_prices = vp.BSM(F, K, days, IV, r, cp_flag)
        
            # original slice where days match (idx: od_TTM -> od)
            idx = od_TTM.index

            # modify the original DataFrame in place
            od.loc[idx, "IV_option_price"] = od_TTM.apply(
                lambda row: vp.BSM(
                    F,
                    row["strike_price"],
                    TTM,
                    row["impl_volatility"],
                    r,
                    row["cp_flag"]
                ),
                axis=1
            )
            od.loc[idx, "moneyness"] = np.log10(F / od_TTM["strike_price"])
            
            
    # Print progress
    elapsed_time = time.time() - start_time
    sys.stdout.write(f"\rRunning date: {current_date} | Tickers: {len(tickers_date)} | Time elapsed: {elapsed_time:.2f} seconds")
    sys.stdout.flush()